In [2]:
# 
abstract type AbstractIndex end

struct Index <: AbstractIndex
    lookup::Dict{Symbol, Int}
    names::Vector{Symbol}
end

function Index(names::AbstractVector{Symbol}; makeunique::Bool=false)
    u = make_unique(names, makeunique=makeunique)
    lookup = Dict{Symbol, Int}(zip(u, 1:length(u)))
    return Index(lookup, u)
end

Index() = Index(Dict{Symbol, Int}(),Symbol[])
Base.length(x::Index) = length(x.names)
Base.names(x::Index) = string.(x.names)

In [ ]:
abstract type AbstractDataFrame end
struct DataFrame <: AbstractDataFrame
    columns::Vector{AbstractVector}
    colindex::Index
    
    #内部构造器
    function DataFrame(columns::Union{Vector{Any}, Vector{AbstractVector}},
        colindex::Index; copycols::Bool=true)
        if length(columns) == length(colindex) == 0
            return new(AbstractVector[], Index())
        elseif length(columns) != 

In [ ]:
# 测试用格子
DataFrame()

In [3]:
Index()

MethodError: MethodError: no method matching Index()
Closest candidates are:
  Index(!Matched::Dict{Symbol,Int64}, !Matched::Array{Symbol,1}) at In[2]:5
  Index(!Matched::Any, !Matched::Any) at In[2]:5